# Overview
- I was inspired by [this article](https://huggingface.co/spaces/HuggingFaceH4/blogpost-scaling-test-time-compute) written by HuggingFace. So, I try this approch into this competition.
- Many inference code come from [Search-And-Learn/HuggingFace Repo](https://github.com/huggingface/search-and-learn).

# Approach
- This approach use main LLM + PRM(Process Reward Model).
- Give LLM enough time to think!
- I use same PRM([RLHFlow/Llam3.1-8B-PRM-Deepseek-Data](https://huggingface.co/RLHFlow/Llama3.1-8B-PRM-Deepseek-Data)) in article.
- And, I used the same System Prompt with a little modification.

# What i tried.
- `QwQ-32B-Preview`, `Qwen2.5-Math-72B-Instruct-AWQ` model. -> CUDA_OUT_OF_MEMORY appeare sometimes.
- Larger Beam number(`n`) and `num_iterations` -> It takes took long.

# Conclusion
I thought this approach was very ingenious. But, The performance was not as good as expected and my best LB Score is QwQ-32B-Preview solo model yet.

There may be some missed code lines or wrote incorrectly. So, All the pointing outs and new ideas are welcome!

In [1]:
import os
import gc
import re
import math
import copy
import time
import logging
import numpy as np
import pandas as pd
import polars as pl

from dataclasses import dataclass
from itertools import accumulate
from typing import Literal, List
from collections import defaultdict, Counter


import torch
import torch.nn.functional as F

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModel,
    PreTrainedModel,
    PreTrainedTokenizer,
    set_seed
)
from tqdm import tqdm

from vllm import LLM, SamplingParams

import kaggle_evaluation.aimo_2_inference_server

In [2]:
cutoff_time = time.time() + (4 * 60 + 30) * 60

In [3]:
def clean_memory(deep=False):
    gc.collect()
    if deep:
        ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

In [4]:
@dataclass
class Config:
    # Chat template related options
    # system_prompt: str = "Please reason step by step.\n\n- For simple problems (2 steps or fewer):\nProvide a concise solution with minimal explanation.\n\n- For complex problems (3 steps or more):\nUse this step-by-step format:\n\n## Step 1: [Concise description]\n[Brief explanation and calculations]\n\n## Step 2: [Concise description]\n[Brief explanation and calculations]\n\n...\n\nRegardless of the approach, always conclude with:\n\nTherefore, the final answer within $\\boxed{}$"
    # system_prompt: str = "You are an expert mathematician. Follow these steps to solve the math problem:\n1. Carefully analyze the problem and break it down into smaller parts.\n2. Solve each part step-by-step, explaining your reasoning clearly and logically.\n3. Use proper mathematical formulas and techniques where necessary.\n4. Double-check your solution for accuracy, and refine it if necessary.\n5. Write your final answer clearly, enclosing it within \boxed{}."
    system_prompt: str = "Please reason step by step, and put your final answer within \\boxed{}." # Qwen2.5 System Prompt
    custom_chat_template: str = None
    # Search Related Options
    n: int = 2**5
    temperature: float = 1.0
    # temperature: float = 0.0
    top_p: float = 0.90
    prm_batch_size: int = 32
    search_batch_size: int = 25
    seed: int = 42
    max_tokens: int = 2048
    agg_strategy: str = "prod"  # Options: "last", "min", "prod"

    # DVTS / Beam Search options
    beam_width: int = 4  # m in the paper
    num_iterations: int = 15
    lookahead: int = 1

    def __post_init__(self):
        self.n_beams = self.n // self.beam_width
        # self.system_prompt = " The answer must be a number, which is the final result of the calculation, without any mathematical symbolic representation."


@dataclass
class Beam:
    prompt: str
    index: int
    current_text: str | None
    next_texts: list[str] | None
    lookahead_texts: list[str] | None
    stop_reasons: list[str | None] | None
    best_scores: list[float]  # the PRM scores
    all_scores: list[list[float]]  # all PRM scores
    previous_text: str | None
    pruned: False
    history: list[str]
    completed: bool = False
    completion_tokens: int = 0


@dataclass
class GenResult:
    index: int
    initial_prompt: str
    first_step_text: str
    first_step_stop_reason: str
    lookahead_text: str
    stop_reason: str | None


    
class PRM:
    def __init__(self, **model_kwargs):
        self.model, self.tokenizer = self.load_model_and_tokenizer(**model_kwargs)

    def load_model_and_tokenizer(
        self, **model_kwargs
    ) -> tuple[PreTrainedModel, PreTrainedTokenizer]:
        raise NotImplementedError

    def score(
        self, questions: list[str], outputs: list[list[str]]
    ) -> list[list[float]]:
        raise NotImplementedError


class RLHFFlow(PRM):
    def load_model_and_tokenizer(
        self, **model_kwargs
    ) -> tuple[PreTrainedModel, PreTrainedTokenizer]:
        tokenizer = AutoTokenizer.from_pretrained(
            "/kaggle/input/rlhflow-llama-8b-prm-deepseek-data/pytorch/base/1"
        )
        model = AutoModelForCausalLM.from_pretrained(
            "/kaggle/input/rlhflow-llama-8b-prm-deepseek-data/pytorch/base/1",
            device_map="auto",
            torch_dtype=torch.bfloat16,
            **model_kwargs,
        ).eval()
        tokenizer.padding_side = "right"
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id

        plus_tag_id = tokenizer.encode("+")[-1]
        minus_tag_id = tokenizer.encode("-")[-1]
        self.candidate_tokens = [plus_tag_id, minus_tag_id]

        return model, tokenizer

    def score(
        self,
        questions: list[str],
        outputs: list[list[str]],
        batched: bool = True,
        batch_size=8,
    ) -> list[list[float]]:
        if batched is True:
            return self._score_batched(questions, outputs, batch_size=batch_size)
        else:
            return self._score_single(questions, outputs)

    def _score_single(self, questions: list[str], outputs: list[list[str]]):
        # reference code: https://github.com/RLHFlow/RLHF-Reward-Modeling/blob/main/math-rm/prm_evaluate.py
        all_scores = []
        for question, answers in zip(questions, outputs, strict=True):
            all_step_scores = []
            for ans in answers:
                single_step_score = []
                conversation = []
                ans_list = ans.split("\n\n")
                for k in range(len(ans_list)):
                    if k == 0:
                        # TODO: add the system prompt like we did for math shepard?
                        text = question + " " + ans_list[0]
                    else:
                        text = ans_list[k]
                    conversation.append({"content": text, "role": "user"})
                    conversation.append({"content": "+", "role": "assistant"})
                    input_ids = self.tokenizer.apply_chat_template(
                        conversation, return_tensors="pt"
                    ).to(self.model.device)
                    with torch.no_grad():
                        logits = self.model(input_ids).logits[
                            :, -3, self.candidate_tokens
                        ]  # simple version, the +/- is predicted by the '-3' position
                        step_scores = logits.softmax(dim=-1)[
                            :, 0
                        ]  # 0 means the prob of + (1 mean -)
                        # print(scores)
                        single_step_score.append(
                            step_scores[0]
                            .detach()
                            .to("cpu", dtype=torch.float32)
                            .item()
                        )

                all_step_scores.append(single_step_score)
            all_scores.append(all_step_scores)
        return all_scores

    def _score_batched(
        self, questions: list[str], outputs: list[list[str]], batch_size: int = 2
    ):
        # The RLHFlow models are trained to predict the "+" or "-" tokens in a dialogue, but since these are not unique
        # we need to introduce a dummy special token here for masking.

        special_tok_id = self.tokenizer("ки", return_tensors="pt").input_ids[0, 1]
        # We construct two parallel dialogues, one with a "+" token per assistant turn, the other with the dummy token "ки" for masking
        conversations = []
        conversations2 = []
        for question, answers in zip(questions, outputs, strict=True):
            for ans in answers:
                conversation = []
                conversation2 = []
                ans_list = ans.split("\n\n")
                for k in range(len(ans_list)):
                    if k == 0:
                        text = question + " " + ans_list[0]
                    else:
                        text = ans_list[k]
                    conversation.append({"content": text, "role": "user"})
                    conversation.append({"content": "+", "role": "assistant"})

                    # we track to location of the special token with ки in order to extract the scores
                    conversation2.append({"content": text, "role": "user"})
                    conversation2.append({"content": "ки", "role": "assistant"})

                conversations.append(conversation)
                conversations2.append(conversation2)

        output_scores = []
        for i in range(0, len(conversations), batch_size):
            convs_batch = conversations[i : i + batch_size]
            convs2_batch = conversations2[i : i + batch_size]
            inputs_batch = self.tokenizer.apply_chat_template(
                convs_batch, padding=True, return_tensors="pt"
            ).to(self.model.device)
            inputs2_batch = self.tokenizer.apply_chat_template(
                convs2_batch, padding=True, return_tensors="pt"
            ).to(self.model.device)
            assert inputs_batch.shape == inputs2_batch.shape
            
            with torch.no_grad():
                logits = self.model(inputs_batch).logits[:, :, self.candidate_tokens]
                scores = logits.softmax(dim=-1)[
                    :, :, 0
                ]  # 0 means the prob of + (1 mean -)

                for i in range(len(convs_batch)):
                    # We slice on the N-1 token since the model is trained to predict the Nth one ("+" in this case)
                    step_scores_flat = scores[i, :-1][
                        inputs2_batch[i, 1:] == special_tok_id
                    ].tolist()
                    output_scores.append(step_scores_flat)

        # reshape the output scores to match the input
        reshaped_output_scores = []
        counter = 0
        for question, answers in zip(questions, outputs):
            scores = []
            for answer in answers:
                scores.append(output_scores[counter])
                counter += 1
            reshaped_output_scores.append(scores)

        return reshaped_output_scores

class QwenPRM:
    def __init__(self, **model_kwargs):
        self.model, self.tokenizer = self.load_model_tokenizer(**model_kwargs)
        self.step_sep = "<extra_0>"
        self.step_sep_id = self.tokenizer.encode(self.step_sep)[0]

    def load_model_tokenizer(self, **model_kwargs):
        tokenizer = AutoTokenizer.from_pretrained(
            "/kaggle/input/qwen2.5-math/transformers/qwen2.5-math-prm-7b/1",
            trust_remote_code=True
        )
        model = AutoModel.from_pretrained(
            "/kaggle/input/qwen2.5-math/transformers/qwen2.5-math-prm-7b/1",
            device_map="auto",
            torch_dtype=torch.bfloat16,
            trust_remote_code=True,
            **model_kwargs,
        ).eval()
        tokenizer.padding_side = "right"
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id
        
        return model, tokenizer

    def make_step_rewards(self, logits, token_masks):
        probabilities = F.softmax(logits, dim=-1)
        probabilities = probabilities * token_masks.unsqueeze(-1) # bs, seq_len, num_labels
        
        all_scores_res = []
        for i in range(probabilities.size(0)):
            sample = probabilities[i] # seq_len, num_labels
            positive_probs = sample[sample != 0].view(-1, 2)[:, 1] # valid_tokens, num_labels
            non_zero_elements_list = positive_probs.cpu().tolist()
            all_scores_res.append(non_zero_elements_list)
        return all_scores_res

    def score(
        self,
        system_prompt: str,
        questions: list[str],
        responses: list[list[str]],
        batched: bool = True,
        batch_size=8,
    ) -> list[list[float]]:
        if batched is True:
            return self._score_batched(system_prompt, questions, responses, batch_size=batch_size)
        else:
            return self._score_single(system_prompt, questions, responses)
            
    def _score_single(
        self,
        system_prompt: str,
        questions: List[str],
        responses: List[List[str]]
    ):
        all_scores = []
        for question, answers in zip(questions, responses, strict=True):
            for ans in answers:
                ans_list = ans.split("\n\n")
                conversation = [
                    {"role" : "system", "content" : system_prompt},
                    {"role": "user", "content": question},
                    {"role": "assistant", "content": self.step_sep.join(ans_list) + self.step_sep}
                ]
                
                input_ids = self.tokenizer.apply_chat_template(
                    conversation, return_tensors="pt"
                ).to(self.model.device)
                
                with torch.no_grad():
                    logits = self.model(input_ids)
                    token_masks = (input_ids == self.step_sep_id)
                    step_reward = self.make_step_rewards(logits[0], token_masks)
                    all_scores.append(step_reward[0])
        return all_scores

    def _score_batched(
        self,
        system_prompt: str,
        questions : List[str],
        responses: List[List[str]],
        batch_size: int
    ):
        all_scores = []
        conversations = []

        for question, answers in zip(questions, responses, strict=True):
            for ans in answers:
                ans_list = ans.split("\n\n")
                conversations.append([
                    {"role" : "system", "content" : system_prompt},
                    {"role": "user", "content": question},
                    {"role": "assistant", "content": self.step_sep.join(ans_list) + self.step_sep}
                ])

        output_scores = []
        for i in range(0, len(conversations), batch_size):
            convs_batch = conversations[i : i + batch_size]
            inputs_batch = self.tokenizer.apply_chat_template(
                convs_batch, padding=True, return_tensors="pt"
            ).to(self.model.device)
            
            with torch.no_grad():
                logits = self.model(inputs_batch)
                token_masks = (inputs_batch == self.step_sep_id)
                scores = self.make_step_rewards(logits[0], token_masks)

                output_scores.extend(scores)

        # reshape the output scores to match the input
        reshaped_output_scores = []
        counter = 0
        for question, answers in zip(questions, responses):
            scores = []
            for answer in answers:
                scores.append(output_scores[counter])
                counter += 1
            reshaped_output_scores.append(scores)

        return reshaped_output_scores
    


In [5]:
def aggregate_scores(
    scores: list[float], agg_strategy: Literal["min", "prod", "last"]
) -> float:
    if agg_strategy == "min":
        return min(scores)
    elif agg_strategy == "prod":
        return math.prod(scores)
    elif agg_strategy == "last":
        return scores[-1]
    else:
        raise ValueError(f"Invalid aggregation strategy: {agg_strategy}")
        

def build_conv(
    prompt: str, response: str | None, system_prompt: str
) -> list[dict[str, str]]:
    conversation = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]

    if response != "":
        conversation.append({"role": "assistant", "content": response})

    return conversation

In [6]:
# llm_model_path = "/kaggle/input/qwen2.5-math/transformers/qwen2.5-math-72b-instruct-awq/1"
llm_model_path = "/kaggle/input/m/qwen-lm/qwen2.5-math/transformers/7b-instruct/1"
llm = LLM(
    llm_model_path,
    # dtype="bfloat16",                # The data type for the model weights and activations
    dtype="half" if "awq" in llm_model_path.lower() else "bfloat16",
    max_num_seqs=256,              # Maximum number of sequences per iteration. Default is 256
    # max_model_len=8192,          # Model context length
    trust_remote_code=True,      # Trust remote code (e.g., from HuggingFace) when downloading the model and tokenizer
    tensor_parallel_size=4,      # The number of GPUs to use for distributed execution with tensor parallelism
    # tensor_parallel_size=2,
    gpu_memory_utilization=0.6, # The ratio (between 0 and 1) of GPU memory to reserve for the model
    # seed=SEED,
)
# prm = RLHFFlow(load_in_4bit=True)
prm = QwenPRM(load_in_4bit=True)

INFO 01-17 17:11:02 config.py:905] Defaulting to use mp for distributed inference
INFO 01-17 17:11:02 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='/kaggle/input/m/qwen-lm/qwen2.5-math/transformers/7b-instruct/1', speculative_config=None, tokenizer='/kaggle/input/m/qwen-lm/qwen2.5-math/transformers/7b-instruct/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execut

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

  self.pid = os.fork()


(VllmWorkerProcess pid=355) INFO 01-17 17:11:04 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
(VllmWorkerProcess pid=357) INFO 01-17 17:11:04 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
(VllmWorkerProcess pid=356) INFO 01-17 17:11:04 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
INFO 01-17 17:11:05 utils.py:1008] Found nccl from library libnccl.so.2
INFO 01-17 17:11:05 pynccl.py:63] vLLM is using nccl==2.20.5
(VllmWorkerProcess pid=357) (VllmWorkerProcess pid=355) (VllmWorkerProcess pid=356) INFO 01-17 17:11:05 utils.py:1008] Found nccl from library libnccl.so.2
INFO 01-17 17:11:05 utils.py:1008] Found nccl from library libnccl.so.2
INFO 01-17 17:11:05 utils.py:1008] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=357) (VllmWorkerProcess pid=355) (VllmWorkerProcess pid=356) INFO 01-17 17:11:05 pynccl.py:63] vLLM is using nccl==2.20.5
INFO 01-17 17:11:05 pynccl.py:63] vLLM is using nccl==2.20.5
INFO 01-17 17:11:05 pynccl.py:63] vLL

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 01-17 17:12:05 model_runner.py:1067] Loading model weights took 3.5478 GB
(VllmWorkerProcess pid=355) (VllmWorkerProcess pid=356) INFO 01-17 17:12:05 model_runner.py:1067] Loading model weights took 3.5478 GB
INFO 01-17 17:12:05 model_runner.py:1067] Loading model weights took 3.5478 GB
(VllmWorkerProcess pid=357) INFO 01-17 17:12:05 model_runner.py:1067] Loading model weights took 3.5478 GB
INFO 01-17 17:12:07 distributed_gpu_executor.py:57] # GPU blocks: 36350, # CPU blocks: 18724
INFO 01-17 17:12:07 distributed_gpu_executor.py:61] Maximum concurrency for 4096 tokens per request: 141.99x
INFO 01-17 17:12:12 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=355) (VllmWorkerProcess pid=357) (VllmWorkerProcess pid=356) INFO 01-17 17:12:12 model_runner.py:1399] CUDA graphs can take additiona

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at /kaggle/input/qwen2.5-math/transformers/qwen2.5-math-prm-7b/1 were not used when initializing Qwen2ForProcessRewardModel: ['lm_head.weight']
- This IS expected if you are initializing Qwen2ForProcessRewardModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Qwen2ForProcessRewardModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
def generate_k_steps(
    templated_convs,
    lookahead_steps: int,
    llm: LLM,
    sampling_params: SamplingParams,
    beam_width: int,
) -> list[Beam]:
    gen_results = []
    for i, text in enumerate(templated_convs):
        for j in range(
            beam_width
        ):  # beam_width
            gen_result = GenResult(
                index=i,
                initial_prompt=text,
                first_step_text="",
                lookahead_text="",
                stop_reason=None,
                first_step_stop_reason=None,
            )
            gen_results.append(gen_result)

    gen_sampling_params = copy.deepcopy(sampling_params)

    for i in range(lookahead_steps + 1):  # lookahead
        if i == 1:  # if lookahead == 1, greedy Generation
            gen_sampling_params.temperature = 0.0  # greedy for the rest of the steps
        # get all generations that did not finish with eos
        current_gen = [  # Get Generations that stop_reason isn't "EOS"
            gen_results[i] for i in range(len(gen_results)) if gen_results[i].stop_reason != "EOS"
        ]
        gen_prompts = [  # Genearte from previous result
            gen_result.initial_prompt + gen_result.lookahead_text for gen_result in current_gen
        ]
        
        llm_outputs = llm.generate(gen_prompts, gen_sampling_params, use_tqdm=False)
        for gen_result, output in zip(current_gen, llm_outputs):
            gen_text = output.outputs[0].text
            # print(f"===lookahead {i}===")
            # print(f"Current_gen :\n{gen_result}\nGen_text :\n{gen_text}")
            if gen_text.strip(' \n') == "":
                print("Generation Text has only break lines.")
                continue
            
            if i == 0:  # If First step, save the generation in "first_step_text"
                gen_result.first_step_text = gen_text
                gen_result.first_step_stop_reason = output.outputs[0].stop_reason  # save stop_reason
                if gen_result.first_step_stop_reason is None:  # If stop_reason is None, EOS
                    gen_result.first_step_stop_reason = "EOS"
            
            gen_result.lookahead_text = gen_result.lookahead_text + gen_text
            gen_result.stop_reason = output.outputs[0].stop_reason
            if gen_result.stop_reason is None:
                gen_result.stop_reason = "EOS"

    outputs: list[Beam] = []

    counter = 0
    for i, text in enumerate(templated_convs):
        next_texts = []
        stop_reasons = []
        lookahead_texts = []
        for j in range(beam_width):
            gen_result = gen_results[counter]
            next_texts.append(gen_result.first_step_text)  
            lookahead_texts.append(gen_result.lookahead_text)
            stop_reasons.append(gen_result.first_step_stop_reason)
            counter += 1

        beam_result = Beam(
            prompt=text,
            index=i,
            current_text="",
            next_texts=next_texts,
            lookahead_texts=lookahead_texts,
            stop_reasons=stop_reasons,
            best_scores=[0.0],
            all_scores=[],
            previous_text=None,
            pruned=False,
            history=[],
        )
        outputs.append(beam_result)

    return outputs

In [8]:
def _dvts(batch_of_prompts: list[str], config: Config, llm: LLM, prm: PRM):
    # Initialize sampling parameters for text generation
    sampling_params = SamplingParams(
        temperature=config.temperature,
        max_tokens=2048,
        min_p=0.1,
        top_p=config.top_p,
        stop=["\n\n"],  # we consider that a step in the problem is indicated by a double newline
        include_stop_str_in_output=True,
        n=1,
    )

    # Initialize beams for each prompt
    beams: list[Beam] = []
    for prompt in batch_of_prompts:
        for i in range(config.n_beams):  # n_beams = n // bean_width, Create independent each Beams.
            beams.append(
                Beam(
                    prompt=prompt,
                    index=i,
                    current_text="",
                    next_texts=None,
                    lookahead_texts=None,
                    best_scores=[0.0],
                    all_scores=[],
                    previous_text=None,
                    pruned=False,
                    stop_reasons=None,
                    history=[],
                )
            )

    # Perform beam search iterations
    for i in tqdm(
        range(config.num_iterations), desc="Beam search iterations"
    ):  # The EOS token comes out, or is repeatedly created until num_iteration is reached.
        # Filter out pruned beams
        gen_beams = [b for b in beams if not b.pruned]
        if len(gen_beams) == 0:
            break

        # Adjust sampling parameters for the last iteration
        if i == config.num_iterations - 1:  # Last Iteration, exclude 'stop token' option
            sampling_params = SamplingParams(
                temperature=config.temperature,
                max_tokens=2048,
                min_p=0.1,
                top_p=config.top_p,
                n=1,
            )

        # Build conversation context for each beam
        convs = [
            build_conv(b.prompt, b.current_text, config.system_prompt) for b in gen_beams
        ]  # Create conversation context for each beam
        continue_final_message = (
            i > 0
        )# If i is greater than 0, continue_final_message is set to True. If this option is True, erase EOS_TOKEN and strip it, which is for the purpose of proceeding the next iteration using the result of the previous iteration. However, I think there will be a problem with generating correctly, so I fix it as False for now.
        add_generation_prompt = (
            i == 0
        )  # If i is 0, set add_generation_prompt to True

        # Get tokenizer and apply custom chat template if provided
        tokenizer = llm.get_tokenizer()
        len_eos_token = len(tokenizer.eos_token)
        
        if config.custom_chat_template is not None:
            tokenizer.chat_template = config.custom_chat_template
        templated_convs = tokenizer.apply_chat_template(  # Apply chat template
            convs,
            add_generation_prompt=add_generation_prompt,
            continue_final_message=continue_final_message,
            tokenize=False,
        )
        # if not continue_final_message and not add_generation_prompt: # To replace the continue_final_message option, manually set it to erase only up to EOS_TOKEN
        #     templated_convs = [context[:-(len_eos_token+1)] for context in templated_convs]
        
        # Generate next steps for each beam
        lookahead = (
            0 if i == config.num_iterations - 1 else config.lookahead
        )
        gen_results = generate_k_steps(templated_convs, lookahead, llm, sampling_params, config.beam_width)

        # Collect prompts and completions for scoring
        prompts, completions = [], []
        for beam, gen_result in zip(gen_beams, gen_results, strict=True):
            beam.next_texts = gen_result.next_texts
            beam.stop_reasons = gen_result.stop_reasons
            beam.lookahead_texts = gen_result.lookahead_texts
            if len(beam.next_texts) != config.beam_width:  # If beam.next_texts and beam_width are not equal, the beam is pruned
                beam.pruned = True
                logger.warning(f"beam {beam.index} has {len(beam.next_texts)} completions")
            prompts.append(beam.prompt)  # prompt를 저장
            completions.append([beam.current_text + t for t in beam.lookahead_texts])

        # Score the generated completions
        all_scores = prm.score(config.system_prompt, prompts, completions, batch_size=2)  # Score PRM

        # Update beams with the best scoring completions
        for beam, scores in zip(
            gen_beams, all_scores, strict=True
        ):  # For each beam, select the generation with the highest score
            agg_scores = [
                aggregate_scores(s, config.agg_strategy) for s in scores
            ]  # aggregate PRM score
            best_score_ind = np.argmax(agg_scores)  # Select the index of the generation with the highest score
            beam.all_scores = scores
            beam.previous_text = beam.current_text 
            beam.current_text = (
                beam.current_text + beam.next_texts[best_score_ind]
            )  # Update the current text with the completion with the highest score
            beam.history.append(beam.next_texts[best_score_ind])
            beam.best_scores = scores[best_score_ind]  # Save highest score index
            if (
                beam.next_texts[best_score_ind] == "" or beam.stop_reasons[best_score_ind] == "EOS"
            ):  # If the generation with the highest score contains an EOS token, the beam is pruned.
                beam.pruned = True

        # Prune beams containing specific text
        for beam in gen_beams:
            if "boxed{" in beam.current_text:
                beam.pruned = True
        clean_memory()

    # Copy results from the last iteration into beam_width beams
    output: list[Beam] = []
    for beam in beams:
        for i in range(config.beam_width):
            output.append(
                Beam(
                    prompt=beam.prompt,
                    index=beam.index,
                    current_text=beam.previous_text + beam.next_texts[i],
                    next_texts=None,
                    lookahead_texts=None,
                    stop_reasons=None,
                    best_scores=beam.all_scores[i],
                    all_scores=beam.all_scores,
                    previous_text=beam.current_text,
                    pruned=beam.pruned,
                    history=beam.history,
                )
            )
    
    return output

In [9]:
ref = pd.read_csv("/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv")
ref['problem'] = ref['problem'] + " At last, What is the final result modulo by 1000?"
problems = ref.problem

In [10]:
problems[3]

'Find the three-digit number $n$ such that writing any other three-digit number $10^{2024}$ times in a row and $10^{2024}+2$ times in a row results in two numbers divisible by $n$. At last, What is the final result modulo by 1000?'

In [11]:
set_seed(42)

In [12]:
# tokenizer = llm.get_tokenizer()
# convs = [build_conv(problems[3],"", "Please reason step by step, and put your final answer within \\boxed{}.")] * 10
# inputss = tokenizer.apply_chat_template(convs, tokenize=False, add_generation_prompt=True)

In [13]:
# sampling_params = SamplingParams(
#                 temperature=1.0,
#                 max_tokens=2048,
#                 min_p=0.1,
#                 top_p=0.95,
#                 n=1,
#             )
# outputs = llm.generate(inputss,sampling_params)

In [14]:
# outputs = [outputs[i].outputs[0].text for i in range(len(outputs))]
# scores = prm.score(system_prompt="Please reason step by step, and put your final answer within \\boxed{}.",
#                    questions=[problems[3]],
#                    responses=[outputs],
#                   batch_size=2)

In [15]:
# beam_results = _dvts(problems.tolist(), config, llm, prm)

In [16]:
def select_beam_result(beam_results, config):
    # TODO: Complete beam Select function
    results = {"completions" : [], "pred" : [], "completion_tokens": [], "scores" : []}
    agg_scores = [aggregate_scores(b.best_scores, config.agg_strategy) for b in beam_results]
    max_agg_score = max(agg_scores)
    # results["completions"].append([b.current_text for b in beam_results])
    # results["scores"].append([b.best_scores for b in beam_results])

    for i, score in enumerate(agg_scores):
        if score == max_agg_score:
            results["pred"].append(
                beam_results[i].current_text
            )

    return results["pred"]

In [17]:
def extract_boxed_text(text):
    pattern = r'\\boxed\{((?:[^\{\}]*|\{(?:[^\{\}]*|\{.*?\})*\})*)\}'
    matches = re.findall(pattern, text)
    if not matches:
        return ""
    return matches[0]

In [18]:
def select_answer(answers):
    results = Counter()

    for answer in answers:
        try:
            answer = int(answer) % 1000
            results[answer] += 1
        except ValueError:
            results[100] += 1
    _, answer = sorted([(v,k) for k,v in results.items()], reverse=True)[0]
    return answer

In [19]:
def predict(id_: pl.DataFrame, question: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
    id_ = id_.item(0)
    print("------")
    print(id_)
    config = Config()
    question = question.item(0)
    question += " At last, calculate the final result modulo by 1000."
    if time.time() > cutoff_time:
        answer = 100
    else:
        answer_beams = _dvts([question], config, llm, prm)
        answer_contexts = select_beam_result(answer_beams, config)
        if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
            pass
        else:
            for context in answer_contexts:
                print(context, end="\n========\n")
        print(f"Question:\n{question}")
        answers = [extract_boxed_text(answer_context) for answer_context in answer_contexts]
        print(f"All Extracted Answers : {answers}")
        answer = select_answer(answers)
        print(f"Predited Answer:\n{answer}")
        print("------\n\n\n")
    print(f"Limit Time left : {cutoff_time-time.time()}")
    return pl.DataFrame({'id': id_, 'answer': answer})

In [20]:
pd.read_csv(
    '/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv'
).drop('answer', axis=1).to_csv('reference.csv', index=False)

In [21]:
inference_server = kaggle_evaluation.aimo_2_inference_server.AIMO2InferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
#             '/kaggle/input/ai-mathematical-olympiad-progress-prize-2/test.csv',
            'reference.csv',
        )
    )

------
192e23


Beam search iterations: 100%|██████████| 15/15 [11:57<00:00, 47.82s/it]


To determine the number of ways to arrange the first round of a tennis tournament with \(4048\) players (including Fred and George) such that Fred and George do not have to play each other, we can follow these steps:

1. **Calculate the total number of ways to arrange all players without any restrictions:**
   - We have \(4048\) players, and we need to pair them up.
   - The number of ways to choose the first pair is \(\binom{4048}{2}\).
   - After the first pair is chosen, we have \(4046\) players left, and the number of ways to choose the second pair is \(\binom{4046}{2}\).
   - This process continues until we have no players left.
   - However, the order in which we choose the pairs does not matter, so we need to divide by the number of ways to arrange \(2024\) pairs, which is \(2024!\).

   Therefore, the total number of ways to arrange all players is:
   \[
   \frac{(4048)!}{(2!)^{2024} \cdot 2024!}
   \]

2. **Calculate the number of ways to arrange all players such that Fred and

Beam search iterations: 100%|██████████| 15/15 [04:25<00:00, 17.68s/it]


To determine the greatest positive integer \( d \) for which there will be \( d \) consecutive days without a flight from Dodola island, we need to find the least common multiple (LCM) of the intervals between the departures of the three airlines. The intervals are 100 days, 120 days, and 150 days.

First, we find the prime factorizations of the numbers:
\[
100 = 2^2 \times 5^2
\]
\[
120 = 2^3 \times 3 \times 5
\]
\[
150 = 2 \times 3 \times 5^2
\]

The LCM is found by taking the highest power of each prime that appears in these factorizations:
\[
\text{LCM} = 2^3 \times 3^1 \times 5^2
\]

Now, we calculate the LCM:
\[
2^3 = 8
\]
\[
3^1 = 3
\]
\[
5^2 = 25
\]

Multiplying these together:
\[
\text{LCM} = 8 \times 3 \times 25
\]

First, multiply 8 and 3:
\[
8 \times 3 = 24
\]

Next, multiply 24 and 25:
\[
24 \times 25 = 600
\]

Thus, the LCM of 100, 120, and 150 is 600. This means that every 600 days, all three airlines will have a flight on the same day.

To find the greatest number of co

...

To find the three-digit number \( n \) such that writing any other three-digit number \( 10^{2024} \) times in a row and \( 10^{2024}+2 \) times in a row results in two numbers divisible by \( n \), we need to analyze the structure of these numbers.

Let's denote the three-digit number by \( a \). Then, the number formed by writing \( a \) \( 10^{2024} \) times in a row is:
\[ N_1 = a \cdot \frac{10^{3 \cdot 10^{2024}} - 1}{9} \]
Similarly, the number formed by writing \( a \) \( 10^{2024}+2 \) times in a row is:
\[ N_2 = a \cdot \frac{10^{3 \cdot (10^{2024}+2)} - 1}{9} = a \cdot \frac{10^{3 \cdot 10^{2024} + 6} - 1}{9} \]

Since \( N_1 \) and \( N_2 \) are both divisible by \( n \), their difference must also be divisible by \( n \):
\[ N_2 - N_1 = a \cdot \left( \frac{10^{3 \cdot 10^{2024} + 6} - 1}{9} - \frac{10^{3 \cdot 10^{2024}} - 1}{9} \right) = a \cdot \frac{10^{3 \cdot 10^{2024} + 6} - 10^{3 \cdot 10^{2024}}}{9} = a \cdot \frac{10^{3 \cdot 10^{2024}} (10^6 - 1)}{9} = a \cdot \

Beam search iterations: 100%|██████████| 15/15 [06:17<00:00, 25.14s/it]


To determine which integers \( n \) in the range \( 2 \leq m \leq 40 \) are artificial, we need to find if there exist \( n \) different positive integers \( a_1, a_2, \ldots, a_n \) such that

\[
a_1 + a_2 + \cdots + a_n = G(a_1, \ldots, a_n) + 1,
\]

where \( G(a_1, \ldots, a_n) \) is the sum of the \(\frac{n(n-1)}{2}\) pairwise greatest common divisors of \( a_1, a_2, \ldots, a_n \).

First, consider the case when \( n = 2 \). Here, \( G(a_1, a_2) = \gcd(a_1, a_2) \), and the equation becomes

\[
a_1 + a_2 = \gcd(a_1, a_2) + 1.
\]

Let \( \gcd(a_1, a_2) = d \). Then \( a_1 = dx \) and \( a_2 = dy \) for some coprime integers \( x \) and \( y \). The equation transforms to

\[
dx + dy = d + 1 \implies d(x + y) = d + 1 \implies d(x + y - 1) = 1.
\]

Since \( d \) is a positive integer, the only solution is \( d = 1 \) and \( x + y - 1 = 1 \), which gives \( x + y = 2 \). The only pair of coprime positive integers that satisfy this is \( (1, 1) \), but \( a_1 \) and \( a_2 \) must be d

Beam search iterations: 100%|██████████| 15/15 [06:50<00:00, 27.37s/it]

To find the greatest possible length of segment \(CD\), where \(D\) is the foot of the perpendicular from \(C\) to the line \(AB\) in triangle \(ABC\) with \(AB = 120\) and circumradius \(R = 100\), we start by using the relationship between the circumradius \(R\) and the sides of the triangle. The circumradius \(R\) of a triangle can be expressed in terms of its sides \(a\), \(b\), and \(c\) and its area \(K\) as follows:
\[
R = \frac{abc}{4K}
\]
Here, let \(a = BC\), \(b = CA\), and \(c = AB = 120\). The area \(K\) of triangle \(ABC\) can also be expressed using the base \(AB = 120\) and the height \(CD\):
\[
K = \frac{1}{2} \times AB \times CD = \frac{1}{2} \times 120 \times CD = 60 \times CD
\]
Substituting this into the circumradius formula, we get:
\[
R = \frac{a \cdot b \cdot 120}{4 \cdot 60 \cdot CD} = \frac{a \cdot b \cdot 120}{240 \cdot CD} = \frac{a \cdot b}{2 \cdot CD}
\]
Given that \(R = 100\), we can substitute this value into the equation:
\[
100 = \frac{a \cdot b}{2 \cd